In [1]:
print('Hello World')

Hello World


In [1]:
import argparse, torch, random
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [2]:

def load_model(model_id: str):
    bnb = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )
    tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb,
        device_map="auto",
        trust_remote_code=False,
    )
    model.eval()
    return tok, model

In [3]:
def generate(tok, model, prompt, max_new_tokens=128, temperature=0.7):
    dev = model.device
    inputs = tok(prompt, return_tensors="pt").to(dev)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            pad_token_id=tok.eos_token_id,
        )
    return tok.decode(out[0], skip_special_tokens=True)

In [4]:
def converse(prompt):
    model = "unsloth/mistral-7b-instruct-v0.3"

    tok, model = load_model(model)
    print(generate(tok, model, prompt))

converse('Hello, is anyone there?')

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

ImportError: 
 requires the protobuf library but it was not found in your environment. Check out the instructions on the
installation page of its repo: https://github.com/protocolbuffers/protobuf/tree/master/python#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.
